# Test: EtherNet/IP Kommunikation mit dem Sensor

Dieses Notebook dient zum Testen der grundlegenden EtherNet/IP-Kommunikation mit dem Mettler Toledo M800 Sensor. 

**Ziele:**
1. Eine Verbindung zum Sensor herstellen.
2. Die Seriennummer des Geräts auslesen, um die Verbindung zu verifizieren.
3. Die konfigurierten Messkanäle auslesen.

Stellen Sie sicher, dass die notwendigen Bibliotheken installiert sind, indem Sie `pip install -r requirements.txt` in Ihrem Terminal ausführen.

### 1. Import und Initialisierung

In [ ]:
import sys
import os
import logging
import yaml

# Fügt das 'src'-Verzeichnis zum Python-Pfad hinzu, damit wir unsere Module importieren können
sys.path.append(os.path.abspath('src'))

from etherip_client import EtherIPClient

# Logging konfigurieren
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# Lade Konfiguration aus config.yaml
print("Lade Konfiguration aus config.yaml...")
with open("config.yaml", 'r') as f:
    config = yaml.safe_load(f)

eth_config = config['ethernetip']
ip_address = eth_config['ip_address']
eds_file = eth_config['eds_file']

print(f"Gelesene Konfiguration: IP={ip_address}, EDS={eds_file}")

print("Initialisiere EtherIPClient...")
# Initialisiere den Client mit den Werten aus der Konfigurationsdatei
client = EtherIPClient(ip_address=ip_address, eds_file=eds_file)

print(f"Client für Gerät mit IP {client.ip_address} wird verwendet.")

### 2. Verbindung herstellen

In [ ]:
print("Verbinde mit dem Gerät...")
client.connect()

if client.driver and client.driver.connected:
    print("Verbindung erfolgreich hergestellt!")
else:
    print("Verbindung konnte nicht hergestellt werden. Bitte überprüfen Sie die IP-Adresse und die Netzwerkverbindung.")

### 3. Geräteinformationen auslesen

Jetzt lesen wir die Geräteinformationen direkt aus dem 'Identity Object' (Klasse `0x01`) des Geräts. Dazu verwenden wir eine generische CIP-Nachricht (`generic_message`), um die Attribute für Seriennummer (`6`) und Produktname (`7`) abzufragen.

In [ ]:
import struct

if client.driver and client.driver.connected:
    try:
        print("Lese Geräteinformationen...")
        
        # Service 0x0E: Get_Attribute_Single
        # Class 0x01: Identity Object
        # Instance: 1

        # Attribut 6: Seriennummer lesen
        sn_result = client.driver.generic_message(
            service=0x0E,
            class_code=0x01,
            instance=1,
            attribute=6
        )
        
        # Attribut 7: Produktname lesen
        pn_result = client.driver.generic_message(
            service=0x0E,
            class_code=0x01,
            instance=1,
            attribute=7
        )

        if sn_result and sn_result.value and pn_result and pn_result.value:
            print(f"DEBUG: Raw Product Name bytes: {pn_result.value}")
            print(f"DEBUG: Raw Serial Number bytes: {sn_result.value}")

            # --- Produktname (SHORT_STRING) ---
            # Das erste Byte ist die Länge, der Rest ist der String.
            product_name = pn_result.value[1:].decode('latin-1').strip()
            
            # --- Seriennummer (UDINT/DWORD) ---
            # Die Seriennummer ist oft eine 32-bit Zahl (UDINT).
            serial_number_display = ""
            if len(sn_result.value) == 4:
                # Wir wissen die Endianness (Byte-Reihenfolge) nicht, also zeigen wir beide.
                sn_little_endian = struct.unpack('<L', sn_result.value)[0]
                sn_big_endian = struct.unpack('>L', sn_result.value)[0]
                serial_number_display = f"{sn_big_endian} (Big-Endian) oder {sn_little_endian} (Little-Endian)"
            else:
                # Fallback, falls es doch ein String ist
                serial_number_display = f"Konnte nicht als 32-bit Zahl dekodiert werden, Roh-String: {sn_result.value.decode('latin-1').strip()}"

            print("\nGeräteinformationen erfolgreich gelesen:")
            print(f"  - Produkt: {product_name}")
            print(f"  - Seriennummer: {serial_number_display}")
            print("\nBitte vergleichen Sie die angezeigte(n) Seriennummer(n) mit dem Aufkleber auf dem Gerät.")

        else:
            error_status = sn_result.error or pn_result.error or 'Unbekannt'
            print(f"Fehler beim Lesen der Geräteinformationen: Keine oder unvollständige Daten empfangen. Status: {error_status}")

    except Exception as e:
        print(f"Fehler beim Lesen der Geräteinformationen: {e}")

### 4. Messkanäle auslesen

Jetzt testen wir die im `EtherIPClient` implementierte Methode, um die Messwerte der konfigurierten Kanäle auszulesen.

In [ ]:
if client.driver and client.driver.connected:
    try:
        print("\nLese Messkanäle...")
        readings = client.read_all_channels()
        if readings:
            print("Aktuelle Messwerte:")
            for channel, value in readings.items():
                print(f"  - {channel}: {value}")
        else:
            print("Konnte keine Messwerte lesen. Die Methode hat keine Daten zurückgegeben.")
    except Exception as e:
        print(f"Fehler beim Lesen der Messkanäle: {e}")

### 5. Kanal-Status auslesen

Jetzt lesen wir den Status für alle Kanäle. Die Logik zur Interpretation der Status-Bits befindet sich nun direkt im `EtherIPClient`.

In [ ]:
if client.driver and client.driver.connected:
    try:
        print("\nLese Kanal-Status...")
        statuses = client.read_channel_statuses()

        if statuses:
            print("Aktueller Kanal-Status:")
            for channel, status_string in statuses.items():
                print(f"  - {channel}: {status_string}")
        else:
            print("Konnte keinen Kanal-Status lesen.")
    except Exception as e:
        print(f"Fehler beim Lesen des Kanal-Status: {e}")

### 6. Verbindung trennen

In [ ]:
if client.driver and client.driver.connected:
    print("\nSchließe die Verbindung.")
    client.driver.close()
    print("Verbindung geschlossen.")